In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
65,"Most movies I can sit through easily, even if ...",negative
697,So you think a talking parrot is not your cup ...,positive
474,I really don't understand who this movie is ai...,negative
145,**WARNING: POSSIBLE SPOILER**<br /><br />If yo...,negative
314,"This is easily the most disappointing, least g...",negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
65,movie sit easily even particularly like movie ...,negative
697,think talking parrot cup tea huh well think ag...,positive
474,really understand movie aimed at absurdity it ...,negative
145,warning possible spoiler br br if get extremel...,negative
314,easily disappointing least gratifying movie en...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    255
positive    245
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
65,movie sit easily even particularly like movie ...,0
697,think talking parrot cup tea huh well think ag...,1
474,really understand movie aimed at absurdity it ...,0
145,warning possible spoiler br br if get extremel...,0
314,easily disappointing least gratifying movie en...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
import mlflow
import dagshub

# 1. Set the tracking URI first - make sure there are no spaces in the URL string
mlflow.set_tracking_uri('https://dagshub.com/akhilcjdata/Mlops_Project.mlflow')

# 2. Initialize dagshub connection
dagshub.init(repo_owner='akhilcjdata', repo_name='Mlops_Project', mlflow=True)

# 3. Now set the experiment
mlflow.set_experiment("Logistic Regression Baseline")

Initialized MLflow to track repo "akhilcjdata/Mlops_Project"

Repository akhilcjdata/Mlops_Project initialized!

2025/05/06 16:07:52 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/e7ee29e58dea42d081a3605b1bf5cee9', creation_time=1746527841418, experiment_id='0', last_update_time=1746527841418, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [17]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-05-06 16:10:00,297 - INFO - Starting MLflow run...
2025-05-06 16:10:00,979 - INFO - Logging preprocessing parameters...
2025-05-06 16:10:02,016 - INFO - Initializing Logistic Regression model...
2025-05-06 16:10:02,017 - INFO - Fitting the model...
2025-05-06 16:10:02,042 - INFO - Model training complete.
2025-05-06 16:10:02,043 - INFO - Logging model parameters...
2025-05-06 16:10:02,377 - INFO - Making predictions...
2025-05-06 16:10:02,379 - INFO - Calculating evaluation metrics...
2025-05-06 16:10:02,385 - INFO - Logging evaluation metrics...
2025-05-06 16:10:03,782 - INFO - Saving and logging the model...
2025/05/06 16:10:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-05-06 16:10:11,268 - INFO - Model training and logging completed in 10.29 seconds.
2025-05-06 16:10:11,268 - INFO - Accuracy: 0.6
2025-05-06 16:10:11,269 - INFO - Precision: 0.

🏃 View run nebulous-skink-999 at: https://dagshub.com/akhilcjdata/Mlops_Project.mlflow/#/experiments/0/runs/a2f61a2ac4b2486a9aef5a9fd19657bc
🧪 View experiment at: https://dagshub.com/akhilcjdata/Mlops_Project.mlflow/#/experiments/0
